In [1]:
import torch
import numpy as np


In [2]:
t1=torch.tensor(4.0)
t1

tensor(4.)

In [7]:
t1.dtype

torch.float32

In [17]:
t2=torch.tensor([1.,2,3,4])
t2

tensor([1., 2., 3., 4.])

In [22]:
t2.dtype

torch.float32

In [27]:
t3=torch.tensor([[5.,6],
                 [7,8],
                 [9,10]])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [35]:
t4=torch.tensor([
    [[11,12,13],
     [13,14,15]],
     [[15,16,17],
      [17,18,19]]
])
t4

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])

In [40]:
print(t1)
t1.shape

tensor(4.)


torch.Size([])

In [45]:
print(t2)
t2.shape

tensor([1., 2., 3., 4.])


torch.Size([4])

In [50]:
print(t3)
t3.shape

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])


torch.Size([3, 2])

In [55]:
print(t4)
t4.shape

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])


torch.Size([2, 2, 3])

In [56]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [57]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [58]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [59]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.2183, -0.6468, -1.5781],
        [-0.6386, -0.8137, -0.3669]], requires_grad=True)
tensor([-0.3387,  0.0942], requires_grad=True)
